# 作業

### 嘗試用 keras 的 DepthwiseConv2D 等 layers 實做 Inverted Residual Block.
   - depthwise's filter shape 爲 (3,3), padding = same
   - 不需要給 alpha, depth multiplier 參數
   - expansion 因子爲 6

##### 載入套件

In [3]:
from keras.models import Input, Model
from keras.layers import DepthwiseConv2D, Conv2D, BatchNormalization, ReLU, Add

Using TensorFlow backend.


##### 定義 Separable Convolution 函數

In [4]:
def InvertedRes(input, expansion):
    '''
    Args:
        input: input tensor
        expansion: expand filters size
    Output:
        output: output tensor
    '''
    # Expansion Layer
    x = Conv2D(expansion * 3, (1,1), padding='same')(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Depthwise Convolution
    x = DepthwiseConv2D((3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Pointwise Convolution
    x = Conv2D(3, (1,1))(x)
    x = BatchNormalization()(x)
    
    # Add Layer
    x = Add()([input, x])
    
    return x

##### 建構模型

In [5]:
input = Input((64, 64, 3))
output = InvertedRes(input, 6)
model = Model(inputs=input, outputs=output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 18)   72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 18)   72          conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 64, 64, 18)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
depthwise_

更多相關連接參考: https://github.com/keras-team/keras-applications/blob/master/keras_applications/mobilenet_v2.py#L425